In [1]:
import pandas as pd
import numpy as np
from hmmlearn import hmm
import theano
import theano.tensor as T
import lasagne
import os
import re

from sklearn.preprocessing import LabelEncoder
from hmmlearn import hmm
from nltk import FreqDist

import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
reviews_df.shape

(27768, 2)

In [2]:
reviews_df = pd.read_csv('reviews.csv', index_col=0)
categories = pd.read_csv('categories.csv', index_col=0)

In [3]:
start_token = '#'
end_token = '&'
reviews = reviews_df['text'].values
reviews = [re.sub("[0-9]", " $", x) for x in reviews]
reviews = [re.sub("[^a-zA-Z$]", " ", x) for x in reviews]
reviews = [review.lower() for review in reviews]
reviews = [review + end_token for review in reviews]
reviews = [start_token + ' [' + str(int(star)) + '] ' + review
           for review, star in zip(reviews, reviews_df.stars.values)]

In [4]:
reviews = reviews[:100]

In [5]:
words = (' '.join(reviews).split())
alphabet = set(words)
le = LabelEncoder()
le.fit(list(alphabet))

seq = le.transform(words)
features = np.fromiter(seq, np.int64)
features = np.atleast_2d(features).T
fd = FreqDist(seq)

In [7]:
frequencies = np.fromiter((fd.freq(i) for i in range(len(alphabet))), dtype=np.float64)
emission_prob = np.stack([frequencies] * 100)

model = hmm.MultinomialHMM(n_components=100, init_params='st', n_iter=100)
model.emissionprob_ = emission_prob

In [8]:
lengths = [len(review.split()) for review in reviews]
model = model.fit(features, lengths)

In [9]:
for i in range(100):
    symbols, _states = model.sample(15)
    print ' '.join(le.inverse_transform(np.squeeze(symbols)))

crushing dinner daughter found & food have drive great it the that cotta a location
which staff the in food ordered for they but ichiban really sunday park is is
the of seating italian it don feature my and but no menu service about be
great jumbo chicken & in the coffee $ know food and mess and not problem
order brews hibachi the advice every dinner share we the customers at would unassuming week
so usually so and down a it god us due were not the that to
pay rice be main casual food t can fans described doubt on scratch said by
food back italian go pleasant throwing dinner is that and up [1] would meal clean
thru main back few for was next the business thing back minutes tables was my
us a to at prepared a had more to and sam guess fed and s
time only order to for valued place i it your this also for the it
love of inlaid home on was was ingredients done noise they was it option had
and gratuity first i with and visiting enjoyed all to s of it love many
most it [5] it i napolis loud 

In [ ]:
output